# Save and Load Model Weights, Serialize Model.


Alright, so here have some code which should feel familiar from previous tutorials, here is what we want to cover:
 1. How to save and load model weights
 2. Save and loading entire model (Serializing model). When we save the entire model it is going to be stored as a data structure, and hence can be loaded on different tensorflow frameworks like tensforflow.js, tensorflow lite etc. You can train the model on your PC and then put in on production on an android app or a web app. Saving the entire model is going to save the following:
    - Saves weights
    - Model architecture
    - Training Configuration (model.compile())
    - Optimizer and states

## 1. Imports and Configuration

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configure GPU memory growth to be dynamic instead of allocating all memory at once
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## 2. Data Loading and Preprocessing

In [2]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

## 3. Model Definition
### 3.1 Sequential API

In [6]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu"),
        layers.Dense(10)
    ]
)

### 3.2 Functional API

In [ ]:
inputs = keras.Input(784)
x = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10)(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

### 3.3 Subclassing

In [9]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(64, activation="relu")
        self.dense2 = layers.Dense(10)

    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)

# SavedModel format or HDF5 format
model3 = MyModel()

# Load the model weights
# model.load_weights('checkpoint_folder/')

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for checkpoint_folder/

## 4. Compile Model

In [7]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

# OR load the entire model, dont need to include the model
# code or the compile code.
# model = keras.models.load_model('saved_model/')

## 5. Model Training and Evaluation

In [8]:
print("Training model...")
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

print("Evaluating model...")
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

print("Saving model/weights...")
"""
The default format of saving the model is the latest h5 format, in the tensorflow
1.0, the default format was HDF5, which needed additional code in the model
to save it, like defining get_config and from_config methods. But in new
formats it is not needed.
"""
model.save_weights('checkpoint_folder/')
# model.save("saved_model/")

Training model...
Epoch 1/2
1875/1875 - 2s - loss: 0.2992 - accuracy: 0.9149
Epoch 2/2
1875/1875 - 1s - loss: 0.1434 - accuracy: 0.9579
Evaluating model...
313/313 - 0s - loss: 0.1293 - accuracy: 0.9600
Saving model/weights...
INFO:tensorflow:Assets written to: saved_model/assets
